In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Read the two CSV files into dataframes
expression = pd.read_csv('../data/raw/OmicsExpressionProteinCodingGenesTPMLogp1.csv')
mutation = pd.read_csv('../data/raw/OmicsSomaticMutations.csv')

/var/folders/nz/rtq9l4590h118vvgvrm2s6dr0000gn/T/ipykernel_47137/1635886351.py:3: DtypeWarning: Columns (22,50,56,57,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  mutation = pd.read_csv('/Users/vittorio/Projects/uni/ml_lab/data/raw/OmicsSomaticMutations.csv')


In [86]:
mutation = mutation[mutation['HugoSymbol'] == 'TP53']  # filter only TP53 mutations
mutation_subset = mutation[['ModelID', 'VariantType']]  # keep only needed columns

expression = expression.rename(columns={expression.columns[0]: 'ModelID'})
expression.drop_duplicates(inplace=True)

# --- Merge datasets ---
merged_df = pd.merge(expression, mutation_subset, on='ModelID', how='left')

# --- Encode mutation type (multiclass target) ---
le = LabelEncoder()
merged_df['VariantType'] = le.fit_transform(merged_df['VariantType'].astype(str))  # Encode NaN as a class if needed

# Save class label mapping
class_names = {index: label for index, label in enumerate(le.classes_)}
print("Class labels:", class_names)

# --- Binary label: is TP53 mutated or not ---
merged_df['Mutated'] = np.where(merged_df['VariantType'] == le.transform(['nan'])[0], 0, 1)  # 0 if VariantType was NaN, else 1
merged_df.columns = merged_df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Drop text in parentheses and strip whitespace


Class labels: {0: 'SNV', 1: 'deletion', 2: 'insertion', 3: 'nan', 4: 'substitution'}


In [88]:
merged_df.to_csv("../data/processed/merged_data.csv")